In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os, itertools, functools, librosa, csv, glob, pickle
import vmo.VMO.utility as utl
import vmo.analysis as van
import numpy as np
import matplotlib.pyplot as plt
import librosa

In [4]:
file_path = '../JKUPDD-Aug2013/groundTruth/'
song_list = ['bachBWV889Fg', 'beethovenOp2No1Mvt3', 'chopinOp24No4', 'gibbonsSilverSwan1612', 'mozartK282Mvt2']
bpm_list = [84, 192, 138, 54, 120]
brange = [[1,110],[-1,555],[-1, 556],[1, 80],[-1,526]]
pattern_list = ['barlowAndMorgensternRevised', 'bruhn','schoenberg','sectionalRepetitions','tomCollins']

def get_occurence(f_path, pattern_list):
    f = itertools.chain.from_iterable([[_f+'/'+c+'/occurrences/csv/' for c in os.listdir(f_path+_f+'/') if not c.startswith('.')] 
                                        for _f in os.listdir(f_path) if (not _f.startswith('.') and _f in pattern_list)])
    f = list(f)
    occur = [[_c for _c in os.listdir(f_path+'/'+_f) if not _c.startswith('.')] for _f in f[:]]
    out = []
    for c,s in zip(occur, f):
        occ_list = []
        for _c in c:
            csv_file = open(f_path+s+_c,'r')
            cs = list(csv.reader(csv_file))
            region = [float(cs[0][0]),float(cs[-1][0])]
            occ_list.append(region)
        out.append(occ_list)
    return out

audio_list = [{'audio':[file_path+song+'/polyphonic/audio/'+a for a in os.listdir(file_path+song+'/polyphonic/audio/') if a.endswith('wav')][0],
               'info':(song, bpm, beat_range),
               'pattern':get_occurence(file_path+song+'/polyphonic/repeatedPatterns/', pattern_list)} 
                for song, bpm, beat_range in zip(song_list, bpm_list, brange)]

In [8]:
fft_size = 8192
hop_size = 128
sr = 11025
feature_dict = {}
for piece in audio_list:
    print 'Process '+piece['info'][0]
    c_vec = []
    b_list = []
    
    y, sr = librosa.load(piece['audio'], sr = sr)
    C = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_size)
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_size)
    
    c_vec.append(C)
    b_list.append(beats)
        
    performance_path = '../performances/'
    p_path =  performance_path+piece['info'][0]+'/'
    perform_list = [f for f in os.listdir(p_path) if f.endswith('mp3')]
    
    for i,p in enumerate(perform_list):
        print '    '+p
        p = p_path+p
        y, sr = librosa.load(p, sr = sr)
        C = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_size)
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_size)

        c_vec.append(C)
        b_list.append(beats)
    piece_dict = {'chroma':c_vec, 'beats':b_list}
    feature_dict[piece['info'][0]] = piece_dict

Process bachBWV889Fg
    bach1.mp3
    bach2.mp3
    bach3.mp3
    bach4.mp3
    bach5.mp3
Process beethovenOp2No1Mvt3
    beethoven1.mp3
    beethoven2.mp3
    beethoven3.mp3
    beethoven4.mp3
    beethoven5.mp3
Process chopinOp24No4
    chopin1.mp3
    chopin2.mp3
    chopin3.mp3
    chopin4.mp3
    chopin5.mp3
Process gibbonsSilverSwan1612
    gibbons1.mp3
    gibbons2.mp3
    gibbons3.mp3
    gibbons4.mp3
    gibbons5.mp3
Process mozartK282Mvt2
    mozart1.mp3
    mozart2.mp3
    mozart3.mp3
    mozart4.mp3
    mozart5.mp3


In [9]:
print feature_dict.keys()

['chopinOp24No4', 'mozartK282Mvt2', 'beethovenOp2No1Mvt3', 'bachBWV889Fg', 'gibbonsSilverSwan1612']


In [10]:
f = open('motif_feature_03132017','w')
pickle.dump(feature_dict, f)
f.close()